In [149]:
%matplotlib inline
import matplotlib as lib
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

# Top Grossing Movies: 
1. Get the data
2. Clean and data
3. Transform to Pandas framework
4. Do calculation if neccessary
5. Explore! 

In [150]:
#Get the data and clean the data
t=requests.get("http://www.the-numbers.com/movie/records/All-Time-Domestic-Box-Office")
soup=BeautifulSoup(t.text)
soup.prettify()
tgm=soup.find_all("tr")
topGrossingMovies={'Title':[],"Year":[],"Domestic":[], "International":[],"Worldwide": []}
for i in range(1, len(tgm)):
    x= tgm[i].find_all("td")
    topGrossingMovies['Year'].append(int(x[1].get_text().strip()))
    topGrossingMovies['Title'].append(x[2].get_text().strip())    
    topGrossingMovies['Domestic'].append(int(x[3].get_text().replace('$','').replace(',','').strip()))
    topGrossingMovies['International'].append(int(x[4].get_text().replace('$','').replace(',','').strip()))
    topGrossingMovies['Worldwide'].append(int(x[5].get_text().replace('$','').replace(',','').strip()))

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [151]:
# Put the data into framework
bdf=pd.DataFrame(topGrossingMovies)
bdf=bdf[['Title','Year','Domestic','International','Worldwide']]
bdf.index=bdf.index +1
bdf.head()

,Title,Year,Domestic,International,Worldwide
1,Star Wars Ep. VII: The Force Awakens,2015,936662225,1122000000,2058662225
2,Avatar,2009,760507625,2023411357,2783918982
3,Titanic,1997,658672302,1548943366,2207615668
4,Jurassic World,2015,652198010,1018130015,1670328025
5,The Avengers,2012,623279547,896200000,1519479547


In [152]:
bdf.to_csv("TopGrossingMovies.csv")

In [153]:
import math
def get_adjusted_price(year, price):
    baseURL="https://www.statbureau.org/calculate-inflation-price-json?country=united-states&start={}/01/01&end=2016/08/30&amount={}"
    price=str(price)
    adjusted=requests.get(baseURL.format(year,price))
    if adjusted.status_code == 200:
        adjustedPrice=float(adjusted.json().strip("$").strip(',').replace(" ",""))
    else:
        adjustedPrice=math.nan
    return adjustedPrice

In [154]:
def wrapper(row):
    return get_adjusted_price(row.Year, row.Domestic)

In [155]:
bdf["Domestic_Adj"]=bdf.apply(wrapper,axis=1)
bdf=bdf[['Title','Year','Domestic','Domestic_Adj','International','Worldwide']]
bdf.sort_values(by=["Domestic_Adj"], ascending=False).head()

,Title,Year,Domestic,Domestic_Adj,International,Worldwide
9,Star Wars Ep. IV: A New Hope,1977,460998007,1.907779e+09,325600000,786598007
90,Jaws,1975,260000000,1.206585e+09,210700000,470700000
13,ET: The Extra-Terrestrial,1982,435110554,1.114869e+09,357854772,792965326
3,Titanic,1997,658672302,1.000272e+09,1548943366,2207615668
1,Star Wars Ep. VII: The Force Awakens,2015,936662225,9.607597e+08,1122000000,2058662225


In [156]:
import math
def get_data(title, year):
    url="http://www.omdbapi.com?t={}&y={}&r=json&plot=short&type=movie"
    title=str(title)
    year=str(year)
    data=requests.get(url.format(title,year))
    if data.status_code == 200:
        data=data.json()
    else:
        data="N/a"
    return data
def get_actors(row):
    data=get_data(row.Title,row.Year)
    if data=="N/a":
        return "N/a"
    elif "Error" in data.keys():
        return "N/a"
    else:
        return data['Actors']
def get_imdbRating(row):
    data=get_data(row.Title,row.Year)
    if data=="N/a":
        return math.nan
    elif "Error" in data.keys():
        return math.nan
    else:
        return data['imdbRating']
def get_Runtime(row):
    data=get_data(row.Title,row.Year)
    if data=="N/a":
        return math.nan
    elif "Error" in data.keys():
        return math.nan
    else:
        return data['Runtime']
def get_Genre(row):
    data=get_data(row.Title,row.Year)
    if data=="N/a":
        return "N/a"
    elif "Error" in data.keys():
        return "N/a"
    else:
        return data['Genre']

In [159]:
data=get_data("Star Wars IV",2014)
data

{'Error': 'Movie not found!', 'Response': 'False'}

In [167]:
pd.set_option('display.max_colwidth', -1)
bdf['Actors']=bdf.apply(get_actors,axis=1)
bdf['Actors'].str.wrap(30,break_long_words=False,break_on_hyphens=False)
bdf['imdbRating']=bdf.apply(get_imdbRating,axis=1)
bdf['Runtime']=bdf.apply(get_Runtime,axis=1)
bdf['Genre']=bdf.apply(get_Genre,axis=1)
bdf=bdf[['Title','Genre','Year',"Actors",'Runtime',"imdbRating",'Domestic','Domestic_Adj',"International",'Worldwide']]
bdf.sort_values(by=["imdbRating","Domestic_Adj"], ascending=False).head()

,Title,Genre,Year,Actors,Runtime,imdbRating,Domestic,Domestic_Adj,International,Worldwide
6,The Dark Knight,"Action, Crime, Drama",2008,"Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine",152 min,9.0,533345358,6.115991e+08,469546000,1002891358
28,The Lord of the Rings: The Return of the King,"Action, Adventure, Drama",2003,"Noel Appleby, Ali Astin, Sean Astin, David Aston",201 min,8.9,377845905,5.030698e+08,763562762,1141408667
47,Forrest Gump,"Comedy, Drama",1994,"Tom Hanks, Rebecca Williams, Sally Field, Michael Conner Humphreys",142 min,8.8,330151138,5.453902e+08,349706026,679857164
53,The Lord of the Rings: The Fellowship of the Ring,"Action, Adventure, Drama",2001,"Alan Howard, Noel Appleby, Sean Astin, Sala Baker",178 min,8.8,315544750,4.367810e+08,571672938,887217688
69,Inception,"Action, Adventure, Sci-Fi",2010,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy",148 min,8.8,292576195,3.263171e+08,540008221,832584416
